In [ ]:
# For usage on google colab:
!git clone https://github.com/ahans30/Binoculars.git
%cd Binoculars
!pip install -e .

In [45]:
from binoculars import Binoculars

ModuleNotFoundError: No module named 'binoculars'

In [46]:
import numpy as np
import torch
import transformers
import pickle as pkl
import pandas as pd
from itertools import islice
from google.colab import files

ModuleNotFoundError: No module named 'google.colab'

In [4]:
bino = Binoculars()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Using pad_token, but it is not set yet.


In [51]:
df = pd.read_csv("/content/raid_test_100.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/raid_test_100.csv'

In [63]:
df.head(40)

,Unnamed: 0,generation,attack,domain,repetition_penalty,model
0,0,Segmentations-Leak: Membership Inference Attac...,perplexity_misspelling,abstracts,no,mistral-chat
1,1,"Farshid Jamshidian is a finance researcher, ac...",insert_paragraphs,wiki,NaN,human
2,2,Transient spine enlargement (3-5 min timesca...,alternative_spelling,abstracts,NaN,human
3,3,The viola da Terceira (also viola Terceiren...,whitespace,wiki,NaN,human
4,4,Yes this review is 2 years after the film's re...,insert_paragraphs,reviews,NaN,human
5,5,Tony Blair is to launch a new campaign to wi...,whitespace,news,no,mpt
6,6,In an underground world where technology reign...,perplexity_misspelling,books,yes,mpt-chat
7,7,"2 cloves garlic, peeled\n5 grams garlic oil\n1...",synonym,recipes,NaN,human
8,8,Men said at vespers: 'All is well!'\nIn one wi...,alternative_spelling,poetry,NaN,human
9,9,"In the realm of verse, a new wave emerges,\nA ...",number,poetry,no,llama-chat


In [53]:
texts = df[['generation']]
print(texts)

                                           generation
0   Segmentations-Leak: Membership Inference Attac...
1   Farshid Jamshidian is a finance researcher, ac...
2     Transient spine enlargement (3-5 min timesca...
3   The   viola da Terceira  (also viola Terceiren...
4   Yes this review is 2 years after the film's re...
..                                                ...
95    We consider the orbit type filtration on a m...
96  George Michael is to perform live at London's ...
97  Nicole Kelly is an Australian contemporary Chr...
98  In this paper, we propose a novel approach to ...
99  If   life is still bad for me even  when I com...

[100 rows x 1 columns]


In [54]:
models = df[['model']]
print(models)

           model
0   mistral-chat
1          human
2          human
3          human
4          human
..           ...
95         human
96         human
97      mpt-chat
98    llama-chat
99         human

[100 rows x 1 columns]


In [9]:
def perplexity(encoding: transformers.BatchEncoding,
               logits: torch.Tensor,
               median: bool = False,
               temperature: float = 1.0):
    shifted_logits = logits[..., :-1, :].contiguous() / temperature
    # print(shifted_logits)
    shifted_labels = encoding.input_ids[..., 1:].contiguous()
    # print(shifted_labels)
    shifted_attention_mask = encoding.attention_mask[..., 1:].contiguous()
    # print(shifted_attention_mask)
    if median:
        ce_nan = (ce_loss_fn(shifted_logits.transpose(1, 2), shifted_labels).
                  masked_fill(~shifted_attention_mask.bool(), float("nan")))
        # print(ce_nan)
        walk = ce_nan.to("cpu").float().numpy()
        # print(walk)
        ppl = np.nanmedian(ce_nan.cpu().float().numpy(), 1)

    else:
        ppl = (ce_loss_fn(shifted_logits.transpose(1, 2), shifted_labels) *
               shifted_attention_mask).sum(1) / shifted_attention_mask.sum(1)
        ppl = ppl.to("cpu").float().numpy()

    return ppl, walk

In [10]:
def entropy(p_logits: torch.Tensor,
            q_logits: torch.Tensor,
            encoding: transformers.BatchEncoding,
            pad_token_id: int,
            median: bool = False,
            sample_p: bool = False,
            temperature: float = 1.0):
    vocab_size = p_logits.shape[-1]
    total_tokens_available = q_logits.shape[-2]
    p_scores, q_scores = p_logits / temperature, q_logits / temperature

    p_proba = softmax_fn(p_scores).view(-1, vocab_size)

    if sample_p:
        p_proba = torch.multinomial(p_proba.view(-1, vocab_size), replacement=True, num_samples=1).view(-1)

    q_scores = q_scores.view(-1, vocab_size)

    ce = ce_loss_fn(input=q_scores, target=p_proba).view(-1, total_tokens_available)
    padding_mask = (encoding.input_ids != pad_token_id).type(torch.uint8)

    if median:
        ce_nan = ce.masked_fill(~padding_mask.bool(), float("nan"))
        en_walk = ce_nan.to("cpu").float().numpy()
        agg_ce = np.nanmedian(ce_nan.cpu().float().numpy(), 1)
        return agg_ce, en_walk
    else:
        agg_ce = (((ce * padding_mask).sum(1) / padding_mask.sum(1)).to("cpu").float().numpy())

    return agg_ce

In [ ]:
# def chunker(it, size):
#     iterator = iter(it)
#     while chunk := list(islice(iterator, size)):
#         print(chunk)
step = 10
batches = [df["generation"].tolist()[x:x+step] for x in range(0, len(df['generation']), step)]
targets = [df["model"].tolist()[x:x+step] for x in range(0, len(df['generation']), step)]
print(batches)
print(targets)

In [ ]:
maxpad = 500 + bino.tokenizer.pad_token_id

bino_walk = []

for b, batch in enumerate(batches):
    print(len(batch))
    encodings = bino._tokenize(batch)
    observer_logits, performer_logits = bino._get_logits(encodings)
    pad_token = bino.tokenizer.pad_token_id
    print(pad_token)

    ce_loss_fn = torch.nn.CrossEntropyLoss(reduction="none")
    softmax_fn = torch.nn.Softmax(dim=-1)

    pf_ppl, pf_walk = perplexity(encodings, performer_logits, median=True)


    x_ppl, en_walk = entropy(observer_logits, performer_logits, encodings, pad_token, median=True)

    if np.array(en_walk).shape[1] > maxpad:
        maxpad = np.array(en_walk).shape[1]
    bino_walk.append([(pf_walk / en_walk[..., :maxpad]), targets[b]])

In [ ]:
bino_walk = np.array(bino_walk)
bino_walk = bino_walk.reshape(100, bino_walk.shape[2])
bino_walk.shape

In [ ]:
pkl.dump(bino_walk, open('bino_walk_list.pkl', 'wb'))
files.download('bino_walk_list.pkl')
bino_walk = pkl.load(open("bino_walk_list.pkl", 'rb'))

In [44]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})
print(bino_walk)

[[0.837209 1.391892 1.129032 ... nan nan nan]
 [1.125561 0.993631 1.051813 ... nan nan nan]
 [2.325843 0.647541 1.462585 ... nan nan nan]
 ...
 [1.037037 0.555944 0.834225 ... nan nan nan]
 [0.471429 0.702312 0.394886 ... nan nan nan]
 [4.724832 1.518672 0.238579 ... nan nan nan]]
